In [421]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd


In [422]:
#FSAPLALONS000005003574,6472722,511,27-05-2022
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [457]:
df = pd.read_csv("data.csv")
loan_account_list = df['Loan-Account'].to_list()
principal_list = df['Principal-M'].to_list()
roi_list = df['Rate-of-interest'].to_list()
emi_list = df['EMI'].to_list()
edd_list = df['Earlier-due-date'].to_list()
# df.head()

In [458]:
part_payment = pd.read_csv("pre-payment.csv")
part_payment['REPAY_EFFECTIVE_DATE'] = pd.to_datetime(part_payment['REPAY_EFFECTIVE_DATE'],format="%d-%m-%Y")
# part_payment.head()

In [459]:
part_payment['date'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).day
part_payment['month'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).month
part_payment['year'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).year
# part_payment.head(8)

In [436]:
part_payment_list=part_payment.loc[part_payment['Loan Account']=="FSAPLALONS000005003574",['month','date','PART_PREPAY_AMOUNT']].values.tolist()
print(part_payment_list)

[[9, 1, 416], [9, 11, 417], [6, 1, 418]]


In [465]:
for index, item in df['Loan-Account'].items():


    part_payment_list=part_payment.loc[part_payment['Loan Account']==item,['month','date','PART_PREPAY_AMOUNT']].values.tolist()
    print(get_part_payment_dict(part_payment_list))
    part_payment_dict = get_part_payment_dict(part_payment_list)
    #APRIL MONTH INTEREST
    principal = first_closing_prinicipal(principal_list[index], roi_list[index], emi_list[index])
    #MAY TO SEPTEMBER INTEREST
    for i in range(5, 10):
        interest_value, closing_principal_2 = next_values(principal, roi_list[index], emi_list[index], edd_list[index], i, i+1, part_payment_dict)
        principal = closing_principal_2
    

{11: [[7, 8], [1000, 2000]]}
{}
{9: [[1, 11], [416, 417]], 6: [[1], [418]]}


In [463]:
def get_part_payment_dict(part_payment_list):
    part_payment_dict = {}

    for line in part_payment_list:
        if line[0] in part_payment_dict:
            # append the new number to the existing array at this slot
            part_payment_dict[line[0]][0].append(line[1])
            part_payment_dict[line[0]][1].append(line[2])
        else:
            # create a new array in this slot
            part_payment_dict[line[0]] = [[line[1]],[line[2]]]
    return part_payment_dict

In [437]:
part_payment_dict = {item[0]: [item[1],item[2:]] for item in part_payment_list}
print(part_payment_dict)

{9: [11, [417]], 6: [1, [418]]}


In [181]:
part_payment_dict.get(9)

[1, [416]]

In [150]:
sm = 9 
if sm == part_payment_dict.get('9-30'):
    print("summa")

416

In [452]:
def days360(start_date, end_date, method_eu=False):
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

In [358]:
# part_payment_dict = {9:[[1],[416]]}
part_payment_dict = {9:[[1,15],[416,1000]]}



In [344]:
part_payment_dict = {9:[[1],[416]]}


In [334]:
i=0
part_payment_dict.get(sm)[0][i+1]
len(part_payment_dict.get(9)[0])


15

2

In [410]:
part_payment_dict = {9:[[1,15],[416,[1000,2000]]]}


In [453]:
# end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
# principal = 35701.08
# roi = 26.73
# emi = 7396
# sm = 9
# part_payment_dict = {9:[[1],[416]]}
# part_payment_dict = {9:[[1,15],[416,1000]]}

def part_payment_interest(end_of_month_dates,principal,roi,emi,part_payment_dict,sm):
    start_date = pd.to_datetime(f"{end_of_month_dates[sm-2]}")

    if len(part_payment_dict.get(sm)[0]) == 1:
        i = 0
        interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i]}'))
        if type(part_payment_dict.get(sm)[1][i]) == list:
            principal = principal - sum(part_payment_dict.get(sm)[1][i])
        else:
            principal = principal - part_payment_dict.get(sm)[1][i]
        interest_value_2 = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i]}'), pd.to_datetime(f"{end_of_month_dates[sm-1]}"))
        interest_value = interest_value_1 + interest_value_2
        principal_apposition = emi - interest_value
        principal = principal - principal_apposition
        return interest_value, principal, principal_apposition
    else:
        end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][0]}')
        interest_value = 0
        for i in range(len(part_payment_dict.get(sm)[0])+1):
            interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f"{end_date}"))
            print("start_date",start_date)
            print("end_date",end_date)
            print("interest_value in loop",interest_value)
            if i==len(part_payment_dict.get(sm)[0]):
                principal = principal - 0
            else:
                if type(part_payment_dict.get(sm)[1][i]) == list:
                    principal = principal - sum(part_payment_dict.get(sm)[1][i])
                else:
                    principal = principal - part_payment_dict.get(sm)[1][i]
            if i==len(part_payment_dict.get(sm)[0])-1:
                end_date = pd.to_datetime(f"{end_of_month_dates[sm-1]}")
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"
            elif i==len(part_payment_dict.get(sm)[0]):
                end_date = pd.to_datetime(f"{end_of_month_dates[sm-1]}")
            else:
                end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i+1]}')
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"


            interest_value = interest_value + interest_value_1
            print("inside loop",interest_value)
        
        principal_apposition = emi - interest_value
        print("principal_apposition",principal_apposition)
        print("interest_value after loop",interest_value)
        principal = principal - principal_apposition
        return interest_value, principal, principal_apposition
    

In [454]:

end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
interest_value_list = []

def first_closing_prinicipal(principal, roi, emi):
    first_interest = principal * (roi/1200)
    interest_value_list.append(first_interest)
    print("first interest",round(first_interest, 2))
    principal_apposition = emi - first_interest
    print("first prinicipal appo",round(principal_apposition, 2))
    closing_principal = principal - principal_apposition
    print("first closing prinicipal",round(closing_principal, 2))
    return closing_principal


def next_values(principal, roi, emi, date, sm, em, part_payment_dict):

    if sm <= 6:
        interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}'))
        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)
        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        if sm==6:
            # print("closing principal inside june before interest", round(principal,2))
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30'))
            interest_value_list.append(interest_value)
            print("inside june interest", round(interest_value,2))
            principal = principal + interest_value
            print("june closing principal", round(principal,2))
    if sm>6:

        # print("closing principal after june before interest", round(principal,2))
        if part_payment_dict.get(sm):
            print("INSIDE PART PAYMENT sm, em", sm, em)
            print("principal before part payment", round(principal,2))
            interest_value, principal, principal_apposition = part_payment_interest(end_of_month_dates,principal,roi,emi,part_payment_dict,sm)
            print("principal after part payment", round(principal,2))
            print("principal apposition after part payment", round(principal_apposition,2))
            print("interest value after part payment", round(interest_value,2))
            

        else:
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f"{end_of_month_dates[sm-2]}"), pd.to_datetime(f"{end_of_month_dates[sm-1]}"))

            #print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
            print("sm, em", sm, em)
            #print("date",date)
            print("interest value", round(interest_value,2))
            interest_value_list.append(interest_value)

            principal_apposition = emi - interest_value
            print("principal apposition", round(principal_apposition,2))
            principal = principal - principal_apposition
            print("closing principal", round(principal,2))
        
    return interest_value,principal

In [455]:
#APRIL MONTH INTEREST
principal = first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])
part_payment_dict = {}
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(principal, roi_list[0], emi_list[0], edd_list[0], i, i+1, part_payment_dict)
    principal = closing_principal_2

first interest 4561.49
first prinicipal appo 4903.51
first closing prinicipal 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
interest value 4348.95
principal apposition 5116.05
closing principal 197711.22
inside june interest 3250.1
june closing principal 200961.32
sm, em 7 8
interest value 4308.95
principal apposition 5156.05
closing principal 195805.26
sm, em 8 9
interest value 4198.39
principal apposition 5266.61
closing principal 190538.66
sm, em 9 10
interest value 4085.47
principal apposition 5379.53
closing principal 185159.12


In [419]:
#APRIL MONTH INTEREST
principal = first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])
part_payment_dict = {}
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(principal, roi_list[0], emi_list[0], edd_list[0], i, i+1, part_payment_dict)
    principal = closing_principal_2

first interest 4561.49
first prinicipal appo 4903.51
first closing prinicipal 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
interest value 4348.95
principal apposition 5116.05
closing principal 197711.22
inside june interest 3250.1
june closing principal 200961.32
sm, em 7 8
interest value 4308.95
principal apposition 5156.05
closing principal 195805.26
sm, em 8 9
interest value 4198.39
principal apposition 5266.61
closing principal 190538.66
sm, em 9 10
interest value 4085.47
principal apposition 5379.53
closing principal 185159.12


In [147]:
print(interest_value_list)

[4561.49, 4456.35, 4348.95, 3250.1, 4308.95, 4198.39, 4085.47]


In [154]:
df['april_interest'] = ""
df['may_interest'] = ""
df['june_interest'] = ""
df['june_interest'] = ""
df['july_interest'] = ""
df['august_interest'] = ""
df['september_interest'] = ""

df['april_interest1'] = ""
df['may_interest1'] = ""
df['june_interest1'] = ""
df['july_interest1'] = ""
df['august_interest1'] = ""
df['september_interest1'] = ""

,Loan Account,Principal-M,Rate-of-interest,EMI,Earlier-due-date,New-due-date,Date,interest_april,interest_april1
0,FDSG898123,212739.43,25.73,9465,7,30,2022-07-31,4561.49,4561.49
1,FDSG898123,37990.00,26.50,5917,7,30,2022-07-31,4561.49,


In [31]:
# decimal - done 
# june  interest value - done
# prepayment logic - done
# prepayment interest calculation - done
# multiple prepayment in same month - done
# same date multiple prepayment - done
# prepayment before june
# same date = end date => part payment case 
# sample data verification